# Restauración de imágenes

## Contenidos:
- Tipos de ruido en imágenes
- Resturación usando máscaras en frecuencia
- Restauración usando filtro inverso
- Restauración usando filtro de Wiener

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider, IntSlider, FloatLogSlider

def color2bw(img):
    return np.dot(img, [0.299, 0.587, 0.114]) 

img_example = color2bw(plt.imread('../images/lobo.jpg')) 

# Ruido en imágenes

Existen distintos tipos de ruido que pueden afectar una imagen

- Ruido térmico, ruido de lectura, ruido eléctronico: Se modelan tipicamente como ruido blanco Gaussiano
- Ruido de disparo (shot noise): Ruido fotónico cuando hay pocas cuentas. Se modela como ruido Poissoniano
- Ruido periódico: Ruido causado por interferencias
- Ruido sal y pimienta: Ruido impulsivo que puede ocurrir por problemas de transmisión o conversión AD


### Ruido blanco Gaussiano

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3.5), tight_layout=True)
def update(intensity, seed):
    ax.cla()
    np.random.seed(seed)
    noise = np.random.randn(img_example.shape[0], img_example.shape[1])
    img_corrupted = img_example + intensity*noise
    ax.matshow(img_corrupted, cmap=plt.cm.Greys_r); 
    ax.axis('off')
interact(update, 
         intensity=FloatLogSlider(min=-1, max=3, description="Intensidad"),
         seed=IntSlider(description="Semilla"));

### Ruido periódico

Tiene frecuencia y dirección

En este ejemplo el ruido tiene una dirección vertical

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3.5), tight_layout=True)
X, Y = np.meshgrid(np.arange(0, img_example.shape[1]), np.arange(0, img_example.shape[0]))

def update(strengh, frequency):
    ax.cla()
    img_corrupted = img_example + strengh*np.cos(2.0*np.pi*frequency*Y/480)
    ax.matshow(img_corrupted, cmap=plt.cm.Greys_r) 
    ax.axis('off')

interact(update, 
         strengh=FloatSlider(min=1, max=100.0, value=50, description="Intensidad"),
        frequency=FloatSlider(min=0.0, max=80, step=0.01, value=0, description="Frecuencia"));

### Ruido impulsivo

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3.5), tight_layout=True)

def update(pbb, seed):
    ax.cla()
    np.random.seed(seed)
    noise = np.random.rand(*img_example.shape)
    img_corrupted  = np.where(noise < pbb, 255, img_example)
    ax.matshow(img_corrupted, cmap=plt.cm.Greys_r) 
    ax.axis('off')

interact(update, 
         pbb=FloatSlider(min=0.0, max=0.5, value=0.0, step=.01),
         seed=IntSlider(description="Semilla"));

# Eliminando ruido periódico usando máscaras en frecuencia

Si entendemos la naturaleza del ruido podemos eliminarlo

Por ejemplo, si el ruido es periódico podríamos intentar "borrar" los componentes frecuenciales asociados al ruido usando una mascara multiplicativa

In [ ]:
from scipy import fftpack

fig, ax = plt.subplots(1, 3, figsize=(8, 6), tight_layout=True);
X, Y = np.meshgrid(np.arange(0, img_example.shape[1]), 
                   np.arange(0, img_example.shape[0]))
freq_x = fftpack.fftshift(fftpack.fftfreq(n=img_example.shape[1]))
freq_y = fftpack.fftshift(fftpack.fftfreq(n=img_example.shape[0]))

def update(frequency, size):
    for ax_ in ax.ravel():
        ax_.cla()
    dims = img_example.shape
    periodic_noise = np.cos(2.0*np.pi*0.235455*Y)
    img_corrupted = img_example + 100*periodic_noise
    ax[0].matshow(img_corrupted[:, 200:500], cmap=plt.cm.Greys_r);
    S_img = fftpack.fftshift(fftpack.fft2(img_corrupted))   
    # Mascaras
    
    freq_int = int(frequency*dims[0])
    S_img[dims[0]//2-size-freq_int:dims[0]//2+size-freq_int, 
          dims[1]//2-size:dims[1]//2+size] = 0 
    S_img[dims[0]//2-size+freq_int:dims[0]//2+size+freq_int, 
          dims[1]//2-size:dims[1]//2+size] = 0 
    # Espectro
    ax[1].imshow(np.log(1+np.abs(S_img))[100:-100, 400:-400], cmap=plt.cm.Spectral_r,
                 extent=(freq_x[400], freq_x[-400], freq_y[100], freq_y[-100]))
    # Reconstrucción
    img_reconstructed = np.mean(img_example) + np.abs(fftpack.ifft2(fftpack.ifftshift(S_img)))
    ax[2].matshow(img_reconstructed[:, 200:500], cmap=plt.cm.Greys_r)

interact(update, 
         size=IntSlider(min=1, max=20, value=10, description="Mask size"),
         frequency=FloatSlider(min=0.0, max=1.0, value=0.04, step=0.01, description="Mask position"));

Causas de degradación de calidad en imágenes
- Manipulación: Desenfoque 
- Ambiente: Reflejos y dispersión de luz
- Dispositivo: Ruido del sensor y circuitos
- Ruido de cuantización


## Restauración de imágenes mediante deconvolución

Una imagen observada $g(x,y)$ se puede modelar como

$$
g(x,y) =  f(x, y) * h(x, y) + n(x,y)
$$

donde 
- $f(x,y)$ es la imagen original
- $n(x,y)$ es ruido blanco aditivo
- $h(x,y)$ es la respuesta al impulso del capturador, también se llama Point Spread Function (PSF)

> La PSF modela las distorsiones causadas por el dispositivo de captura o por el ambiente

### Ejemplo: PSF de un telescopio
<table><tr><td>
    <img src="../images/psf3.png" width="250"></td><td><img src="../images/PSF1.png" width="550">
</td></tr></table>
<center><img src="../images/PSF2.jpeg" width="600"></center>


# Deconvolución en frecuencia: Filtro inverso


Deconvolución se refiere al proceso de recuperar $f(x,y)$ a partir de $g(x,y)$ usando supuestos sobre $h(x,y)$ y $n(x,y)$

Si trabajamos en frecuencia:

$$
G(f_1, f_2) = F(f_1, f_2) \cdot H(f_1, f_2) + N(f_1, f_2)
$$

Si conocemos $H$ e  ignoramos $N$ podemos estimar $F$ usando un **filtro inverso**

$$
\hat F(f_1, f_2) = G(f_1, f_2) / H(f_1, f_2)
$$

>  ¿Problema resuelto?

Simulemos una PSF Gaussiana con $\sigma=2$ y ruido blanco Gaussiano con desviación estándar 20 y estudiemos el filtro inverso

In [ ]:
from scipy.signal import fftconvolve, convolve2d

def minmax_normalize(data):
    return (data - np.amin(data))/(np.amax(data) - np.amin(data))

x = np.linspace(-5, 5, num=11)
X, Y = np.meshgrid(x, x)

sigma = 2
psf = np.exp(-0.5*(X)**2/sigma**2 - 0.5*Y**2/sigma**2)/(2.0*np.pi*sigma**2)

img_example_observed = fftconvolve(img_example, psf, mode='same') + 20*np.random.randn(*img_example.shape)
img_example_observed = 255*minmax_normalize(img_example_observed)

plt.figure(figsize=(8, 5), tight_layout=True)
plt.imshow(img_example_observed, cmap=plt.cm.Greys_r);

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fx = fftpack.fftfreq(n=img_example.shape[1], d=1)
fy = fftpack.fftfreq(n=img_example.shape[0], d=1)
Fx, Fy = np.meshgrid(fx, fy) 
fig, ax = plt.subplots(2, 2, figsize=(8, 5), tight_layout=True);

def update(sigma, tol):
    for ax_ in ax.ravel():
        ax_.cla(); ax_.axis('off');        
    ax[0, 0].matshow(img_example_observed, cmap=plt.cm.Greys_r);
    S_img = fftpack.fft2(img_example_observed/255.)
    ax[0, 1].matshow(fftpack.fftshift(np.log(1.0+np.abs(S_img))), cmap=plt.cm.Spectral_r); 
    # Filtro inverso PSF en frecuencia
    inv_PSF =  np.exp(-2*np.pi**2*(Fx**2 + Fy**2)*sigma**2)
    # Cuidamos de no dividir por cero
    inv_PSF[inv_PSF > 1.0/10**tol] = 0.0
    ax[1, 0].matshow(fftpack.fftshift(inv_PSF), cmap=plt.cm.Spectral_r); 
    # Reconstrucción
    img_recovered = np.real(fftpack.ifft2(S_img*inv_PSF))
    ax[1, 1].matshow(img_recovered, cmap=plt.cm.Greys_r); 
    
interact(update, 
         sigma=FloatSlider(min=0.1, max=10.0, value=0.1, description="Ancho PSF $\sigma$"),
         tol=FloatSlider(min=-3, max=-0.1, value=-5, description="log(tolerance)"));

- El filtro inverso es difícil de calibrar
- Fenomeno de ampligicación de ruido
<img src="../images/noise-amplification.png">

 

# Filtro de Wiener

Sea un filtro lineal para estimar la imagen real a partir de la imagen observada

$$
\hat f(x, y) = w(x,y) * g(x, y)
$$

Podemos buscar $w$ tal que el MSE sea mínimo:

$$
\text{MSE} = \mathbb{E} \left[ \left(f(x,y) - \hat f(x,y) \right)^2 \right]
$$

Podemos resolver la ecuación anterior en frecuencia (asumiendo N ruido de media cero) y obtener

$$
W(f_1, f_2) = \frac{H(f_1, f_2)^{*}}{|H(f_1, f_2)|^2 + \frac{S_n(f_1, f_2)}{S_f(f_1, f_2)}},
$$

donde $S_n = |N(f_1, f_2)|^2$ es la densidad espectral del ruido y $S_f = |F(f_1, f_2)|^2$ es la densidad espectral de la señal original.

- En general no conocemos las densidades espectral de potenica
- El factor $S_f/S_n$ se puede interpretar como la razón señal a ruido (SNR)
- Podemos hacer supuestos sobre la SNR

Notemos también que si $S_n \to 0$ se recupera el filtro inverso

<img src="../images/wiener-noise.png">


In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(8, 5), tight_layout=True);

def update(K):
    for ax_ in ax.ravel():
        ax_.cla(); ax_.axis('off')
    ax[0, 0].matshow(img_example_observed, cmap=plt.cm.Greys_r);
    S_img = fftpack.fft2(img_example_observed/255.)
    # Transformada de Fourier de la PSF
    PSF =  np.exp(-2*np.pi**2*(Fx**2 + Fy**2)*2**2)
    # Filtro de Wiener
    WF = np.conj(PSF)/(np.abs(PSF)**2 + 10**K)
    ax[0, 1].matshow(fftpack.fftshift(np.abs(WF)), cmap=plt.cm.Spectral_r)
    img_recovered = np.real(fftpack.ifft2(S_img*WF))
    ax[1, 0].matshow(img_recovered, cmap=plt.cm.Greys_r);
    ax[1, 1].matshow(img_recovered[80:250, 240:440], cmap=plt.cm.Greys_r);
    
interact(update, K=FloatSlider(min=-5, max=2, value=0, description="log Sn/Sf"));